# Tổng quan phần thu thập dữ liệu
1. Mục tiêu
2. Công cụ sử dụng
3. Các bước thu thập dữ liệu

In [21]:
# import những  thư viện cần thiết
import pandas as pd 
import numpy as np
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import math
import requests
import time
# !pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## Hàm gửi yêu cầu đến server yêu cầu tải toàn bộ nội dung trang web

In [22]:

def get_content(url, headers):
    # time.sleep(1)
    session = HTMLSession()
    response = session.get(url, headers=headers)
    if(response.status_code == 200):
        print("Success!!!")
        return response
    return None

In [23]:
BASE_URL = 'https://vieclam24h.vn'


headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}


## Step 1: Lấy những liên kết của trang web công việc
- Đầu tiên, khi vô một trang web tìm việc, ta sẽ thấy một loạt danh sách các công việc, sau khi nhấn vào từng cái sẽ liên kết trực tiếp đến trang chi tiết công việc. Vậy đầu tiên ta đến trang tìm kiếm và lọc tất cả các đường dẫn đến công việc.

In [24]:
# Code demo request lấy page list công việc
job_pool = pd.DataFrame()
for work_method in range(1,7):
   #https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=6
   url = BASE_URL + '/tim-kiem-viec-lam-nhanh?&working_method={}'.format(work_method)
   print(url)
   page_content = get_content(url, headers)
   soup = BeautifulSoup(page_content.text, 'html.parser')
   
   buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
   buffer2 = buffer1.find('span',class_="font-semibold").text
   n_jobs = int(buffer2.replace(',',''))
   # Trung bình mỗi page có 30 công việc
   n_pages = min(51, math.ceil(n_jobs/30))
   print('Có {} công việc, {} trang ở work_method = {}'.format(n_jobs, n_pages, work_method))

   for i in range(1,2):
      # Khởi tạo url vầ yêu cầu lấy page về 
      url = BASE_URL + '/tim-kiem-viec-lam-nhanh?page={}&working_method={}'.format(i, work_method)
      print(url)
      page_content = get_content(url, headers)
      soup = BeautifulSoup(page_content.text, 'html.parser')
      
      containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
      print('Đã tìm thấy {} công việc'.format(len(containers)))
      for container in containers:
         dict_job = {}
         
         buffer = BASE_URL + container.find('a', href=True)['href']
         dict_job['Liên kết'] = buffer if buffer else None
         
         buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
         dict_job['Tên công việc'] = buffer.text if buffer else None
         
         buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
         dict_job['Tên công ty'] = buffer.text if buffer else None
         
         buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
         dict_job['Mức lương'] = buffer.text if buffer else None
         
         buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
         dict_job['Khu vực tuyển'] = buffer.text if buffer else None
         
         job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool.sample(5)
         


https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?&working_method=1


Success!!!
Có 5645 công việc, 51 trang ở work_method = 1
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=1&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?&working_method=2
Success!!!
Có 68 công việc, 3 trang ở work_method = 2
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=1&working_method=2
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?&working_method=3
Success!!!
Có 125 công việc, 5 trang ở work_method = 3
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=1&working_method=3
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?&working_method=4
Success!!!
Có 25 công việc, 1 trang ở work_method = 4
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=1&working_method=4
Success!!!
Đã tìm thấy 25 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?&working_method=5
Success!!!
Có 43 công việc, 2 trang ở work_method = 5
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
133,https://vieclam24h.vn/ban-hang-kinh-doanh/thuc...,Thực Tập Sinh Kinh Doanh (Tư Vấn Làm Hồ Sơ Cho...,Công Ty Tài Chính Trách Nhiệm Hữu Hạn Một Thàn...,3 - 5 triệu,"Thừa Thiên Huế, Đà Nẵng, Khánh Hòa, Bình Định, +6"
12,https://vieclam24h.vn/ke-toan/ke-toan-tong-hop...,Kế Toán Tổng Hợp - Đi Làm Ngay,Công Ty Cổ Phần Đầu Tư Và Thương Mại Phúc Hưng,10 - 13 triệu,Hà Nội
64,https://vieclam24h.vn/ban-si-ban-le-quan-ly-cu...,Nhân Viên Bán Hàng (Có Hỗ Trợ Chỗ Ở),Gà Ủ Muối Hoa Tiêu An Nhiên,10 - 15 triệu,TP.HCM
169,https://vieclam24h.vn/san-xuat-lap-rap-che-bie...,Thực Tập Sinh Điện Công Nghiệp / Điện Lạnh,Công Ty TNHH SX-TM-DV Nam Cường,5 - 8 triệu,Bình Dương
28,https://vieclam24h.vn/xay-dung/chi-huy-truong-...,Chỉ Huy Trưởng Công Trình,Công Ty CP Hóa Chất Xây Dựng Bách Khoa,13 - 17 triệu,Hà Nội


In [5]:
# Lưu dữ liệu vào file csv
job_pool.to_csv('../data/job_pool.csv', encoding='utf-8-sig', index=False)

# Step 2: Thu thập dữ liệu của từng trang web
- Sau khi thu thập được các link trang web, ta tiến hành thu thập dữ liệu của từng trang web bằng cách sử dụng cách tương tự

In [6]:
# Đọc dữ liệu từ file job_links.csv
df = pd.read_csv('../data/job_pool.csv')
df.sample(5)

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
9,https://vieclam24h.vn/hanh-chinh-thu-ky/bien-p...,Biên Phiên Dịch Tiếng Trung Tại Long An,Công Ty Cổ Phần Gia Phú,10 - 20 triệu,Long An
16,https://vieclam24h.vn/xay-dung/ky-su-trac-dac-...,Kỹ Sư Trắc Đạc Biết Tiếng Anh,Công Ty TNHH TM DV Tổng Hợp Đăng Khoa,13 - 18 triệu,"Nghệ An, Thừa Thiên Huế, Quảng Trị, Quảng Bình"
58,https://vieclam24h.vn/bien-phien-dich/nhan-vie...,통역 채용 공고- Nhân Viên Phiên Dịch Tiếng Hàn,Công Ty TNHH Sinyoung Vina,15 - 25 triệu,"Ninh Bình, Hà Nội"
45,https://vieclam24h.vn/ban-hang-kinh-doanh/truo...,Trưởng Phòng Tuyển Sinh Du Học Nghề,Cen Land - Công Ty Cổ Phần Bất Động Sản Thế Kỷ,40 - 50 triệu,TP.HCM
48,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Nhân Viên Kinh Doanh (HAYA Hotel /Palma Resort...,Công Ty Tnhh Một Thành Viên Du Lịch Phương Nguyễn,7 - 15 triệu,Hà Nội


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Liên kết       60 non-null     object
 1   Tên công việc  60 non-null     object
 2   Tên công ty    60 non-null     object
 3   Mức lương      60 non-null     object
 4   Khu vực tuyển  60 non-null     object
dtypes: object(5)
memory usage: 2.5+ KB


In [8]:
raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển', 
# Thông tin chung    
'Cấp bậc', 'Số lượng tuyển', 'Hình thức làm việc', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])
for url in df['Liên kết']:

    job_content = get_content(url, headers=headers).content
    soup = BeautifulSoup(job_content, 'html.parser')

    dict_job = {}

    dict_job['Liên kết'] = url

    buffer = soup.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
    dict_job['Tên công ty'] = buffer.text if buffer else None

    buffer = soup.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
    dict_job['Tên công việc'] = buffer.text if buffer else None

    buffer = soup.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
    dict_job['Mức lương'] = buffer.text if buffer else None

    buffer = soup.find('a', class_='hover:text-se-accent')
    dict_job['Khu vực tuyển'] = buffer.text if buffer else None


    buffer = soup.find('p', string="Cấp bậc")
    dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Số lượng tuyển")
    dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Hình thức làm việc")
    dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Yêu cầu bằng cấp")
    dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
    dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Ngành nghề")
    dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None


    buffer1 = soup.find('h4', string="Mô tả công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Yêu cầu công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Quyền lợi")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Quyền lợi'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Địa điểm làm việc")
    buffer2 = buffer1.find_next_sibling('ul')
    dict_job['Địa điểm làm việc'] = buffer2.text if buffer2 else None 


    buffer1 = soup.find('h3', string="Từ khoá")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Từ khóa'] = buffer2.text if buffer2 else None

    company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

    buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
    for buffer in buffers:
        if buffer.get_text() == 'Địa chỉ:':
            dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
        if buffer.get_text() == 'Quy mô:':
            dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None

    raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!


In [9]:
raw_job_details.sample(5)

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Cấp bậc,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
47,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Công Ty TNHH Đầu Tư Và Phát Triển Công Nghệ Mới,Nhân Viên Kinh Doanh Thiết Bị Điện (Thu Nhập T...,15 - 30 triệu,Hà Nội,Chuyên viên- nhân viên,3,Toàn thời gian cố định,Cao đẳng,1 năm,Bán hàng - Kinh doanh/Điện - Điện tử - Điện lạ...,"Hà Nội, Ô 15 TT5.1 KĐT Lexington Estate, Hoàn...",- Có kinh nghiệm kinh doanh về thiết bị điện- ...,- Trình độ cao đẳng trở lên.- Ngoại hình ưa nh...,- Thu nhập: Từ 15 - 30 triệu/tháng + % Giá trị...,trungnhậpquận hoàng maiViệc làm Hà NộiBán hàng...,"ô 15 TT5.1, Khu Đô Thị Ao Sào - Phường Thịnh L...",10 - 150 người
23,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Công Ty Cổ Phần Thuốc Thú Y-Thủy Sản Thịnh Phú,Nhân Viên Kinh Doanh Thuốc Thú Y Lương Cứng Từ...,12 - 30 triệu,Hà Nội,Chuyên viên- nhân viên,5,Toàn thời gian cố định,Cao đẳng,Chưa có kinh nghiệm,Bán hàng - Kinh doanh/Nông - Lâm - Ngư nghiệp/...,"Hà Nội, Lô 2, CN6, Cụm CN Ngọc Hồi, Xã Ngọc Hồ...","- Chăm sóc, phát triển hệ thống khách hàng tại...",- Trình độ: Tốt nghiệp chuyên ngành Thú y hoặc...,- Tổng thu nhập: Từ 12 - 30 triệu/ tháng (Lươn...,thuốclươnghuyện ngọc hồiViệc làm Hà NộiBán hàn...,"Lô 2, CN6, Cụm CN Ngọc Hồi, Xã Ngọc Hồi, Huyện...",150 - 300 người
53,https://vieclam24h.vn/co-khi-o-to-tu-dong-hoa/...,Công Ty Cổ Phần Thiết Bị Và Giải Pháp Cơ Khí A...,Trưởng Phòng Thiết Kế Khuôn Jig (Kcn Đình Trám...,30 - 33 triệu,Bắc Giang,Quản lý cấp trung,1,Toàn thời gian cố định,Đại học,5 năm,Cơ khí - Ô tô - Tự động hóa/Điện - Điện tử - Đ...,"Bắc Giang, Làm Việc Tại KCN Đình Trám, Việt Yên","- Thiết kế khuôn, Jig (Khuôn dập liên hoàn, Th...",- Có ít nhất r5năm kinh nghiệm về Vị trí trưởn...,"- LƯƠNG CƠ BẢN: ~ 30,000,000VNĐ- 33.000.000- ...",thiết kế khuôn mẫunhân viên khuôn mẫucViệc làm...,"Làm Việc Tại KCN Đình Trám, Bắc Giang",10 - 150 người
39,https://vieclam24h.vn/ban-hang-kinh-doanh/truo...,Công Ty CP Thang Máy Và Thiết Bị Thành Công,Trưởng Phòng Kinh Doanh,20 - 30 triệu,Hà Nội,Quản lý cấp trung,1,Toàn thời gian cố định,Cao đẳng,2 năm,Bán hàng - Kinh doanh/Bán sỉ - Bán lẻ - Quản l...,"Hà Nội, Số 7A Mạc Thị Bưởi, Phường Vĩnh Tuy, H...","- Quản lý, hỗ trợ nhân viên kinh doanh trainin...",- Tốt nghiệp Cao đẳng / Đại học.- Tối thiểu 2 ...,- Thời gian thử việc: 2 tháng- Độ tuổi: 30-50-...,quận hai bà trưngtrưởng phòng kinh doanh tại đ...,"Số 7A Mạc Thị Bưởi, Phường Vĩnh Tuy, Quận Hai ...",10 - 150 người
56,https://vieclam24h.vn/lao-dong-pho-thong/lao-d...,Công Ty TNHH Yazaki Hải Phòng Việt Nam,Lao Động Phổ Thông (Công Đoạn Sản Xuất) Có Xe ...,7.1 - 8.4 triệu,"Hải Phòng,",Chuyên viên- nhân viên,100,Toàn thời gian cố định,Trung học,Chưa có kinh nghiệm,Lao động phổ thông/Cơ khí - Ô tô - Tự động hóa,"Hải Phòng, Lô đất L, Khu công nghiệp Nhật Bản ...",- Đứng thao tác lắp ghép bộ dây dẫn điện trong...,"- Nam, nữ tuổi từ 18~45- Trình độ: Tốt nghiệp ...",- Thu nhập bình quân:+ Nhà máy Hải Phòng: từ 8...,xexuấtphổ thôngViệc làm Hải PhòngLao động phổ ...,Hải Phòng - Thái Bình - Quảng Ninh,Trên 300 người


In [10]:
# Lưu vào file craw_dataset.csv
raw_job_details.info()
raw_job_details.to_csv('../data/raw_dataset.csv', encoding='utf-8-sig', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Liên kết             60 non-null     object
 1   Tên công ty          60 non-null     object
 2   Tên công việc        60 non-null     object
 3   Mức lương            60 non-null     object
 4   Khu vực tuyển        60 non-null     object
 5   Cấp bậc              60 non-null     object
 6   Số lượng tuyển       60 non-null     object
 7   Hình thức làm việc   60 non-null     object
 8   Yêu cầu bằng cấp     55 non-null     object
 9   Yêu cầu kinh nghiệm  60 non-null     object
 10  Ngành nghề           60 non-null     object
 11  Địa điểm làm việc    60 non-null     object
 12  Mô tả công việc      60 non-null     object
 13  Yêu cầu công việc    60 non-null     object
 14  Quyền lợi            60 non-null     object
 15  Từ khóa              60 non-null     object
 16  Địa chỉ cô

# Một số phần demo để testing
- Đây là đoạn code để người collect data có thể test thử trước khi chạy toàn bộ code

In [20]:
# Demo lất page content
with open('../data/page_content.txt', 'r', encoding='utf-8') as f:
    page_text = f.read()

soup = BeautifulSoup(page_text, 'html.parser')
buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
buffer2 = buffer1.find('span',class_="font-semibold").text
n_jobs = int(buffer2.replace(',',''))
n_pages = min(50, math.floor(n_jobs/30))
n_pages

50

In [11]:
# Demo lất page content
with open('../data/page_content.txt', 'r', encoding='utf-8') as f:
    page_text = f.read()

job_pool = pd.DataFrame(columns=['Liên kết', 'Tên công việc','Tên công ty','Mức lương','Khu vực tuyển'])

soup = BeautifulSoup(page_text, 'html.parser')
containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
print('Đã tìm thấy {} công việc'.format(len(containers)))
for container in containers:
    dict_job = {}
    
    buffer = BASE_URL + container.find('a', href=True)['href']
    dict_job['Liên kết'] = buffer if buffer else None
    
    buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
    dict_job['Tên công việc'] = buffer.text if buffer else None
    
    buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
    dict_job['Tên công ty'] = buffer.text if buffer else None
    
    buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
    dict_job['Mức lương'] = buffer.text if buffer else None
    
    buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
    dict_job['Khu vực tuyển'] = buffer.text if buffer else None
    
    job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool

Đã tìm thấy 30 công việc


,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
0,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Nhân Viên CSKH - Ứng Dụng Du Lịch Tiếng Anh (Q...,Công Ty TNHH Vietnam Concentrix Services,10 - 15 triệu,TP.HCM
1,https://vieclam24h.vn/hanh-chinh-thu-ky/bien-p...,Biên Phiên Dịch Tiếng Trung Tại Long An,Công Ty Cổ Phần Gia Phú,10 - 20 triệu,Long An
2,https://vieclam24h.vn/ban-hang-kinh-doanh/tro-...,Trợ Lý Phát Triển Kinh Doanh,Công Ty Cổ Phần Giấy CP,12 - 14 triệu,TP.HCM
3,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Telesales (Chợ Tốt Xe),Công Ty TNHH Chợ Tốt Trading,6 - 10 triệu,Hà Nội
4,https://vieclam24h.vn/cham-soc-khach-hang/nhan...,Nhân Viên Dịch Vụ Khách Hàng,Mcredit - Công Ty Tài Chính TNHH Mb Shinsei,10 - 20 triệu,Hà Nội
5,https://vieclam24h.vn/ban-hang-kinh-doanh/chuy...,Chuyên Viên Tư Vấn Tài Chính Hồ Chí Minh (Lươn...,Công Ty TNHH Bảo Hiểm Nhân Thọ AIA Việt Nam,16 - 30 triệu,TP.HCM
6,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Kinh Doanh Nội Thất - Thu Nhập 20-30...,Công Ty TNHH Kiến Trúc Milan,20 - 30 triệu,Hà Nội
7,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Trợ Lý (Biết Tiếng Trung),Công Ty TNHH Việt Nam Paiho,7 - 15 triệu,TP.HCM
8,https://vieclam24h.vn/ke-toan/ke-toan-truong-c...,Kế Toán Trưởng,Công Ty TNHH Thương Mại Dịch Vụ Phát Triển Sản...,18 - 30 triệu,TP.HCM
9,https://vieclam24h.vn/ngan-hang/hcm-giam-doc-q...,HCM - Giám Đốc Quan Hệ Khách Hàng Ưu Tiên,Acb - Ngân Hàng Thương Mại Cổ Phần Á Châu,16 - 50 triệu,TP.HCM


In [16]:

# job = get_content('https://vieclam24h.vn/hanh-chinh-thu-ky/bien-phien-dich-tieng-trung-tai-long-an-c1p123id200290693.html',headers= headers)

# with open('../data/job_content.txt', 'w', encoding='utf-8') as f:
#     f.write(str(job.text))

In [12]:
# Demo lấy chi tiết

raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển', 
# Thông tin chung    
'Cấp bậc', 'Số lượng tuyển', 'Hình thức làm việc', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])

# with open('../data/job_content.txt', 'r', encoding='utf-8') as f:
#     job_content = f.read()
url = 'https://vieclam24h.vn/xay-dung/chi-huy-truong-cong-trinh-c31p73id200288561.html'
job_content = get_content(url, headers=headers).text
soup = BeautifulSoup(job_content, 'html.parser')

dict_job = {}

dict_job['Liên kết'] = 'https://vieclam24h.vn/ban-hang-kinh-doanh/'

buffer = soup.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
dict_job['Tên công ty'] = buffer.text if buffer else None

buffer = soup.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
dict_job['Tên công việc'] = buffer.text if buffer else None

buffer = soup.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
dict_job['Mức lương'] = buffer.text if buffer else None

buffer = soup.find('a', class_='hover:text-se-accent')
dict_job['Khu vực tuyển'] = buffer.text if buffer else None


buffer = soup.find('p', text="Cấp bậc")
dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Số lượng tuyển")
dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Hình thức làm việc")
dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Yêu cầu bằng cấp")
dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Yêu cầu kinh nghiệm")
dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Ngành nghề")
dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None


buffer1 = soup.find('h4', text="Mô tả công việc")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Yêu cầu công việc")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Quyền lợi")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Quyền lợi'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Địa điểm làm việc")
buffer2 = buffer1.find_next_sibling('ul')
dict_job['Địa điểm làm việc'] = buffer2.text if buffer2 else None 


buffer1 = soup.find('h3', text="Từ khoá")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Từ khóa'] = buffer2.text if buffer2 else None

company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
for buffer in buffers:
    if buffer.get_text() == 'Địa chỉ:':
        dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
    if buffer.get_text() == 'Quy mô:':
        dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None

raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)
raw_job_details


Success!!!


C:\Users\huynh\AppData\Local\Temp\ipykernel_7408\539015741.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Cấp bậc")
C:\Users\huynh\AppData\Local\Temp\ipykernel_7408\539015741.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Số lượng tuyển")
C:\Users\huynh\AppData\Local\Temp\ipykernel_7408\539015741.py:39: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Hình thức làm việc")
C:\Users\huynh\AppData\Local\Temp\ipykernel_7408\539015741.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Yêu cầu bằng cấp")
C:\Users\huynh\AppData\Local\Temp\ipykernel_7408\539015741.py:45: DeprecationWarning: The 'text' argument to find()-type methods is deprecate

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Cấp bậc,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
0,https://vieclam24h.vn/ban-hang-kinh-doanh/,Công Ty CP Hóa Chất Xây Dựng Bách Khoa,Chỉ Huy Trưởng Công Trình,13 - 17 triệu,Hà Nội,Quản lý cấp trung,2,Toàn thời gian cố định,Đại học,2 năm,Xây dựng/Quản lý dự án,"Hà Nội, Phòng 2904, tháp A, chung cư thăng lon...",- Điều phối chung các công việc tại dự án.- Là...,- Tốt nghiệp đại học- Có chứng chỉ giám sát hi...,- Lương: Cạnh tranh theo năng lực (Thu nhập : ...,công trìnhchỉ huy trưởng công trình tại hà nội...,"Phòng 2904, tháp A, chung cư thăng long Number...",10 - 150 người
